### Convergence Criterium

In [8]:
def getConvergenceEpoch(epochs, losses, patience=3, min_delta=0):
    """
    Convergence Criterium for the pretext task.
    
    :param epochs: (Tupel/Array): The epochs for each given loss value sorted in ascending order. 
    :param losses: (Tupel/Array): The losses for each given epochs. 
    :param patience: (Integer): How many epochs to wait before declaring convergence of now lower loss is found.
    :param min_delta: (Integer): A delta (threshold) for the current value to count as lower value.
    :return: convergence_epoch: (Integer) the epoch of convergence.
    :return: convergence_loss: (Float) the loss at convergence.
    """
    best = 999999
    wait = 0
    for epoch, current in zip (epochs, losses): 
        if np.less(current - min_delta, best):
            best = current
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                return int(epoch), current
    print("No convergence!")        
    return int(epochs[-1]), losses[-1]   

In [18]:
def getConvergenceEpochPerModel(pretext_model_names, p_epochs_per_xFold_per_model,p_measurements_per_xFold_per_model, patience=3, min_delta=0):
    """
    Helper function to calculate the epoch of convergence for each given model.
    
    :param pretext_model_names: (Array of Strings): The pretext model names to calculate the convergence epoch for.
    :param p_epochs_per_xFold_per_model: (Array of Arrays of Tupels/Arrays): The epochs for each given loss value sorted in ascending order for every model and xFold. 
    :param p_measurements_per_xFold_per_model: (Array of Arrays of Tupels/Arrays): The measurments for each given epochs for every model and xFold. 
    :param patience: (Integer): How many epochs to wait before declaring convergence if no lower measurment is found.
    :param min_delta: (Integer): A delta (threshold) for the current measurmet to count as lower measurmet.
    :return: p_converget_epoch_per_model: (Array of Integer) the epoch of convergence for each model.
    """
    p_converget_epoch_per_model = []
    for i in range(len(pretext_model_names)):
        p_epochs = p_epochs_per_xFold_per_model[i][0]
        p_mean_loss = np.mean(p_measurements_per_xFold_per_model[i], axis=0)
        p_converget_epoch, p_convergence_loss = getConvergenceEpoch(p_epochs, p_mean_loss, patience=patience, min_delta=min_delta)
        p_converget_epoch_per_model.append(p_converget_epoch)
        print("Using Convergence at Epoch: " + str(p_converget_epoch) + " with loss: " + str(p_convergence_loss))
    return p_converget_epoch_per_model   

### Linear Interpolation

In [10]:
def linearInterpolateMeasurements(list_of_measurements, measurement_epochs):
    """
    Linear Interpolation of each measurement tupel in the list. Returns interploated values for each epoch and measurement tupel.
    
    :param p_epochs_per_xFold_per_model: (Array of Tupels/Arrays): List of [The epochs for each given measurement sorted in ascending order]. 
    :param p_losses_per_xFold_per_model: (Array of Tupels/Arrays): List of [The measurements for each given epochs]. 
    :return: interpolated_list_of_measurements: (Array of Tupels/Arrays): List of [The interpolated epochs for each given measurement sorted in ascending order]. 
    :return: interpolated_measurement_epochs: (Array of Tupels/Arrays): List of [The interpolated measurements for each given epochs]. 
    """
    interpolated_list_of_measurements = []
    interpolated_measurement_epochs = np.arange(np.min(measurement_epochs), np.max(measurement_epochs)+1).astype(int)
    for measurement in list_of_measurements:
        interpolated_list_of_measurements.append(np.interp(interpolated_measurement_epochs,measurement_epochs,measurement))
    return interpolated_list_of_measurements, interpolated_measurement_epochs

### Metrics Mismatch (M3) and Hard Objective Function Mismatch

In [11]:
def MM3(measurements_metric_1, measurements_metric_2):
    """
    Calculates the mean Metrics Mismatch between measurements of two metrics as described in the paper.
    
    :param measurements_metric_1: (Array of Tupels/Arrays): Measurements sorted the same way as measurements_metric_2.
    :param measurements_metric_2: (Array of Tupels/Arrays): Measurements sorted the same way as measurements_metric_1.
    :return: mm3: (Float): The Mean Metrics Mismatch (MM3) between the two measurement tupels. 
    """
    return np.mean(measurements_metric_1 - measurements_metric_2)

### Soft Metrics Mismatch (S3M) and Soft Objective Function Mismatch (OFM)

In [12]:
def OFMNormalization(measurements_metric_1, shift_measurements=False):
    """
    Calculates the normalization for the Objective Function Mismatch for measurements of a metric for the given tupel.
    
    :param measurements_metric_1: (Array of Tupels/Arrays): Measurements (of a model).
    :param shift_measurements: (Boolean): If true the measurements to lie between 0 and 100.
    :return: norm_measurements_metric_1: (Array of Tupels/Arrays): Normalized measurements (of a model) for OFM calculation.
    """
    if shift_measurements:
        measurements_metric_1 = np.subtract(measurements_metric_1, np.min(measurements_metric_1))
    
    init_measurment = measurements_metric_1[0]
    min_measurment = np.min(measurements_metric_1)  
    
    norm_measurements_metric_1 = []
    denominator = np.subtract(measurements_metric_1[0], np.min(measurements_metric_1))
    for value in measurements_metric_1:
        if (value == init_measurment) and (init_measurment == min_measurment):
            norm_measurements_metric_1.append(0)
        elif (value > init_measurment) and (init_measurment == min_measurment):
            norm_measurements_metric_1.append(float('inf'))
        else:
            norm_measurements_metric_1.append(np.divide(value, denominator))
                     
    return np.multiply(norm_measurements_metric_1, 100)     

In [13]:
def SM3(index, measurements_metric_1):
    """
    Calculates the Soft Metrics Mismatch for a given step/epoch (index) described in the paper.
    
    :param index: (Integer): The step/epoch to calculate SM3 for.
    :param measurements_metric_1: (Array of Tupels/Arrays): Measurements (of a model).
    :return: sm3: (Float): The Soft Metrics Mismatch (SM3) at this step/epoch. 
    """
    return measurements_metric_1[index] - np.min(measurements_metric_1[:index+1])

In [14]:
def MSM3(measurements_metric_1):
    """
    Calculates the Mean Soft Metrics Mismatch between measurements of two metrics as described in the paper.
    
    :param measurements_metric_1: (Array of Tupels/Arrays): Measurements (of a model).
    :return: msm3: (Float): The Mean Soft Metrics Mismatch (MSM3) of the two metrics. 
    """        
    sm3s = [SM3(i, measurements_metric_1) for i in range(len(measurements_metric_1))]
    msm3 = np.mean(sm3s)     
    return msm3

In [15]:
def cSM3(measurements_metric_1):
    """
    Calculates the Convergence Soft Metrics Mismatch between measurements of two metrics as described in the paper.
    
    :param measurements_metric_1: (Array of Tupels/Arrays): Measurements (of a model).
    :return: cSM3: (Float): The Convergence Soft Metrics Mismatch (cSM3) of the two metrics. 
    """
    return SM3(len(measurements_metric_1)-1, measurements_metric_1)

In [16]:
def mSM3(measurements_metric_1):
    """
    Calculates the Maximum Soft Metrics Mismatch between measurements of two metrics as described in the paper.
    
    :param measurements_metric_1: (Array of Tupels/Arrays): Measurements (of a model).
    :return: msm3: (Float): The Maximum Soft Metrics Mismatch (mSM3) of the two metrics. 
    """
    sm3s = [SM3(i, measurements_metric_1) for i in range(len(measurements_metric_1))]
    max_sm3_index = np.argmax(sm3s) 
    return sm3s[max_sm3_index], max_sm3_index

In [19]:
def cSM3AtConvergence(p_epochs_per_xFold, p_measurements_per_xFold, t_epochs_per_xFold, t_measurements_per_xFold, patience, min_delta):
    """
    Calculates the Convergence Soft Metrics Mismatch on the given erros for each individual cross-validation 
    and returns the mean error on convergence as well ass its range.
    
    :param p_epochs_per_xFold: (Array of Tupels/Arrays): The measuring epochs for every corss-validation.
    :param p_losses_per_xFold: (Array of Tupels/Arrays): The corresponding losses to the cpochs for every corss-validation.
    :param p_measurements_per_xFold: (Array of Tupels/Arrays): The corresponding measurments to the epochs of the pretext model for every corss-validation.
    :param t_measurements_per_xFold: (Array of Tupels/Arrays): The corresponding measurments to the epochs of the target model for every corss-validation.
    :param patience: (Integer): How many epochs to wait before declaring convergence if no lower measurment is found.
    :param min_delta: (Integer): A delta (threshold) for the current measurmet to count as lower measurmet.
    :return: mean_csm3: (Float): The mean Convergence Soft Metrics Mismatch (mSM3) of the given metrics.
    :return: mean_csm3_plus: (Float): The positive range of the Convergence Soft Metrics Mismatch (mSM3).
    :return: mean_csm3_minus: (Float): The negative range of the Convergence Soft Metrics Mismatch (mSM3).
    """
    csm3s_per_xFold= []
    for i in range(len(p_epochs_per_xFold)):
        p_converget_epoch = getConvergenceEpoch(p_epochs_per_xFold[i], p_measurements_per_xFold[i], patience, min_delta)
        print("Using Convergence at Epoch: " + str(p_converget_epoch[0]))
        if use_interpolation:
            measurements, epochs = linearInterpolateMeasurements([t_measurements_per_xFold[i]], t_epochs_per_xFold[i])
        p_converget_epoch_index = np.where(epochs == p_converget_epoch[0])[0][0]
        
        measurements = measurements[0][:p_converget_epoch_index+1]
        csm3s_per_xFold.append(cSM3(measurements))
        
    mean_csm3 = np.mean(csm3s_per_xFold)
    mean_csm3_plus = np.max(csm3s_per_xFold) - mean_csm3
    mean_csm3_minus = np.min(csm3s_per_xFold) - mean_csm3
    return mean_csm3, mean_csm3_plus, mean_csm3_minus